In [ ]:
import requests
import pandas as pd
import numpy as np
import urllib.parse



In [ ]:

def ImportGameWeeks(Nlim, DataFolder):

    Gameweeks=0
    dfGW=-1
    for k in range(38,0,-1):
      url="https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/gws/gw{0}.csv".format(k)
      try:
        r = requests.get(url)
        if r.status_code == 200:
          dfGWTMP=pd.DataFrame([x.split(',') for x in r.text.split('\n')])
          headers = dfGWTMP.iloc[0]
          dfGWTMP  = pd.DataFrame(dfGWTMP.values[1:], columns=headers)
          dfGWTMP["weekID"]=k
          if Gameweeks==0:
            dfGW=dfGWTMP
          else:
            dfGW2=pd.concat([dfGW, dfGWTMP])
            dfGW=dfGW2
          Gameweeks+=1
          if Gameweeks == Nlim:
            break
      except Exception as e:
        print("Warning:> Min week extracted {0}!\n{1}".format(k, str(e)))
        break

    dfGW['kickoff_time']=pd.to_datetime(dfGW['kickoff_time'])

    dfGW.dropna(subset = ["element"], inplace=True)
    dfGW = dfGW.astype({'creativity':'float64', 'ict_index':'float64', 'influence':'float64', 'threat':'float64'})
    dfGW = dfGW.astype({'value':'float64', 'total_points':'int64', 'xP':'float64',
                        'assists':'int64', 'bonus':'int64', 'bps':'int64', 
                        'clean_sheets':'int64', 'goals_conceded':'int64', 'goals_scored':'int64' 
                      , 'element':'int64', 'minutes':'float64', 'own_goals':'int64', 'penalties_missed':'int64'
                        , 'penalties_saved':'int64', 'red_cards':'int64', 'saves':'int64'
                        , 'transfers_balance':'int64', 'transfers_out':'int64', "yellow_cards":'int64'
                      })

    aggregations={
      'xP':["sum"],
      "assists":["sum"],	
      "bonus":["sum"],
      "clean_sheets":["sum"],
      "goals_conceded":["sum"],
      "goals_scored":["sum"],
      "ict_index":["sum"],
      "influence":["sum"],
      "minutes":["mean"],
      "own_goals":["sum"],
      "penalties_missed":["sum"],
      "penalties_saved":["sum"],
      "red_cards":["sum"],
      "saves":["sum"],
      "threat":["sum"],
      "total_points":["sum"],
      "transfers_balance":["sum"],
      "transfers_in":["sum"],
      "transfers_out":["sum"],
      "value":["mean"]
    }
    dfGW2=dfGW.groupby(['name', 'position', 'team','element']).agg(aggregations)
    dfGW2.columns=dfGW2.columns.droplevel(1)
    dfGW2=dfGW2.rename_axis(['name', 'position', 'team','element']).reset_index()
    dfGW2.to_parquet("{0}/GWSummary.parquet".format(DataFolder), compression=None)
    

In [ ]:
def ImportPlayersRaw(DataFolder):
  url="https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2021-22/players_raw.csv"
  r = requests.get(url)
  try:
    if r.status_code == 200:
        dfPlayers=pd.DataFrame([x.split(',') for x in r.text.split('\n')])
        headers=dfPlayers.iloc[0]
        dfPlayers=pd.DataFrame(dfPlayers.values[1:], columns=headers)
        #print(dfPlayers)
        dfPlayers.to_parquet("{0}/PlayersRaw.parquet".format(DataFolder), compression=None)
  except Exception as e:
        print("Error:> when extracting players :{0}!\n".format(str(e)))